<a href="https://colab.research.google.com/github/shashenderkumar/ML/blob/master/BBCNewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
data_train = pd.read_csv('BBC News Train.csv')

In [3]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB


In [4]:
data_train.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


## **Split data in train and test**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [6]:
X = data_train['Text'].values
# X = data_train.iloc[0:2].values
y = data_train['Category'].values

y_t = np.array(y)
# integer encode
le = LabelEncoder()
integer_encoded = le.fit_transform(y_t)
# binary encode
ohe = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_t1 = ohe.fit_transform(integer_encoded)
print(y_t1)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y_t1, test_size = 0.10, random_state = 1000)

In [8]:
print(X_train.size)
print(X_test.size)
print(len(data_train))

1341
149
1490


### **Tokenize Data**

In [9]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(X_train)

X_train1 = tokenizer.texts_to_sequences(X_train)
X_test1 = tokenizer.texts_to_sequences(X_test)

In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [12]:
print(vocab_size)

23910


In [13]:
print(X_train[1:6])

['dollar hovers around record lows the us dollar hovered close to record lows against the euro on friday as concern grows about the size of the us budget deficit.  analysts predict that the dollar will remain weak in 2005 as investors worry about the state of the us economy. the bush administration s apparent unwillingness to intervene to support the dollar has caused further concern. however  trading has been volatile over the past week because of technical and automated trading and light demand. this has amplified reactions to news  analysts said  adding that they expect markets to become less jumpy in january.  the dollar was trading at $1.3652 versus the euro on friday morning after hitting a fresh record low of $1.3667 on thursday. one dollar bought 102.55 yen.  disappointing business figures from chicago triggered the us currency s weakness on thursday. the national association of purchasing management-chicago said its manufacturing index dropped to 61.2  a bigger fall than expec

In [14]:
print(X_train1[0])
print(X_train1[1])
print(X_train1[2])
print(X_train1[3])
print(X_train1[4])
print(X_train1[5])

[2888, 97, 448, 8, 164, 1713, 164, 19, 293, 1, 670, 21, 5, 449, 161, 991, 1, 1273, 331, 11, 14, 1, 79, 535, 2, 132, 1644, 453, 2888, 97, 3, 1, 43, 4, 89, 554, 97, 21, 1, 331, 1645, 22, 409, 221, 350, 1680, 1, 151, 3615, 1713, 4185, 164, 726, 1231, 2, 1, 143, 3, 1, 73, 161, 1862, 12, 42, 703, 6, 1067, 72, 1084, 823, 749, 3269, 3783, 1863, 37, 266, 1, 117, 690, 8, 638, 855, 79, 489, 66, 148, 11, 14, 2156, 1144, 22, 2045, 123, 4, 179, 273, 123, 37, 25, 186, 2, 16, 214, 1602, 64, 29, 25, 321, 34, 243, 23, 29, 956, 54, 12, 1, 855, 8, 1, 117, 690, 97, 3, 209, 37, 335, 2, 1, 1274, 535, 1, 2479, 3, 1, 844, 489, 6, 1864, 5, 48, 2781, 6, 323, 161, 4, 1481, 3112, 1998, 11, 824, 2, 4720, 1, 4186, 3, 1, 63, 1345, 164, 14, 1740, 61, 185, 83, 4, 9, 428, 2, 20, 398, 248, 3113, 1, 143, 1865, 8, 1, 89, 297, 97, 3, 1, 43, 335, 2, 1, 3, 4452, 27, 3270, 14, 1562, 1, 2577, 97, 3, 133, 42, 703, 14, 4453, 22, 164, 37, 14, 489, 12, 1, 289, 148, 11, 14, 181, 91, 110, 697, 22, 1527, 4, 14, 2992, 8, 42, 37, 27, 1

**PADS Sequence**

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
# have to check the value of this hyperparameter
maxlen = 1000

In [0]:
X_train1 = pad_sequences(X_train1, padding= 'post', maxlen = maxlen)
X_test1 = pad_sequences(X_test1, padding= 'post', maxlen = maxlen)

In [18]:
print(X_train1[0])

[2888   97  448    8  164 1713  164   19  293    1  670   21    5  449
  161  991    1 1273  331   11   14    1   79  535    2  132 1644  453
 2888   97    3    1   43    4   89  554   97   21    1  331 1645   22
  409  221  350 1680    1  151 3615 1713 4185  164  726 1231    2    1
  143    3    1   73  161 1862   12   42  703    6 1067   72 1084  823
  749 3269 3783 1863   37  266    1  117  690    8  638  855   79  489
   66  148   11   14 2156 1144   22 2045  123    4  179  273  123   37
   25  186    2   16  214 1602   64   29   25  321   34  243   23   29
  956   54   12    1  855    8    1  117  690   97    3  209   37  335
    2    1 1274  535    1 2479    3    1  844  489    6 1864    5   48
 2781    6  323  161    4 1481 3112 1998   11  824    2 4720    1 4186
    3    1   63 1345  164   14 1740   61  185   83    4    9  428    2
   20  398  248 3113    1  143 1865    8    1   89  297   97    3    1
   43  335    2    1    3 4452   27 3270   14 1562    1 2577   97    3
  133 

In [19]:
print(X_train1[1057])
print(X_train1[1058])
print(X_train1[1059])

[1408    2 1699   54   12    1  191   22  652    3    1   73    7 1665
   24  131   16  300  283  510   21  129  120    5 1420  248 1276  155
 2128   24 1699   54   12    5  938  729    3 2359  222   94   18  880
    4 2129  276 3024    1  258   22 1677  739  852   11    2 1172  155
   56  557    3    1   73   46   25   28  412  297    1  772   10    1
  233 4368   24   17   67    9 1911    1  258    1 1420  628   12    1
   76    4 1677  263    2  223   41   17   48 1091    2 4823  150   10
   20   39   22    1  233 2752  312    1 2932    3 2128  300  129  510
   24   20 1776 2002  150   10  744  233   24 1699   54   12   40    1
  258 1147   18   40    4   40 4928  279  297    4  222  381  375    1
 4368   24  381   40 2148    4   40    8  150   10   20   28  265  191
  510   11 3024    1 2250  162   20 2621    4   20   19   39  151 4629
   53   23 3403   53  139   94    5 4368   24 2834  296   19   39  375
 3325  852  108   54    2 2889  296 4619  240   12  392  276    4  351
 1103 

### **Model Creation**

In [0]:
import tensorflow as tf
m = tf.keras.models
layers = tf.keras.layers

In [0]:
model = m.Sequential()

In [22]:
# vocab size is 23910
# input_length is size of review text after tokenization and pad sequance
embedding_dim = 100


model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=maxlen))

model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 100)         2391000   
_________________________________________________________________
flatten (Flatten)            (None, 100000)            0         
_________________________________________________________________
dense (Dense)                (None, 10)                1000010   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 3,391,021
Trainable params: 3,391,021
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
history = model.fit(X_train1, y_train,
                    epochs=20,verbose=True,
                    validation_data=(X_test1, y_test),
                    batch_size=10)

Epoch 1/20


ValueError: ignored